In [104]:
import pandas as pd
from itertools import combinations, combinations_with_replacement
import numpy as np
import csv
import operator

In [2]:
def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller

In [3]:
def urns_and_balls(U, B ,urns_max=False, urns_min=False, urns_names=False, zeros=False):
    """
    Distributes B balls into U urns
    - if zeros == True  : empty urns allowed 
    - if zeros == False : empty urns not allowed 
    """
    if type(urns_max) == bool: urns_max = np.ones(U,dtype=int)*B
    else: urns_max = np.array(urns_max)
    if type(urns_min) == bool: urns_min = np.zeros(U,dtype=int)
    else: urns_max = np.array(urns_max)
        
    if U == 1:
        if type(urns_names) == bool: return [[B]]
        else: return [{urns_names[0]:B}]
        
    if zeros == True:
        C = combinations_with_replacement(range(B+1),U-1)
        a = np.ones(B,dtype=int)
        g = []
        for c in C:
            x = [sum(a[:c[0]])]
            for i in range(len(c)-1): 
                x.append(sum(a[c[i]:c[i+1]]))
            x.append(sum(a[c[-1]:]))
            if all([i <= j for i,j in zip(x,urns_max)]) and all([i >= j for i,j in zip(x,urns_min)]):
                if type(urns_names) == bool: g.append(x)
                else: g.append(dict(zip(urns_names,x)))
    
    if zeros == False:
        if B < U: print(f'WARNING: Number of balls {B} has to be greater than or equal to the number of urns {U}.')
        C = combinations(range(1,B),U-1)
        a = np.ones(B,dtype=int)
        g = []
        for c in C:
            x = [sum(a[0:c[0]])]
            for i in range(len(c)-1): 
                x.append(sum(a[c[i]:c[i+1]]))
            x.append(sum(a[c[-1]:]))
            if all([i <= j for i,j in zip(x,urns_max)]) and all([i >= j for i,j in zip(x,urns_min)]):
                if type(urns_names) == bool: g.append(x)
                else: g.append(dict(zip(urns_names,x)))
    return g

In [56]:
# Load Data 

data = pd.read_csv("rome_tension.csv")

# Define tree

data.sort_values(by=['rome1','rome3','rome5'])
cat = data.drop(columns=['BE_id','T']).drop_duplicates()
tree1 = dict(cat.groupby(by='rome1')['rome3'].unique())
tree3 = dict(cat.groupby(by='rome3')['rome5'].unique())
tree = {key1:{key3:list(tree3[key3]) for key3 in tree1[key1]} for key1 in tree1.keys()}

# Delete these groups: they are too big!! 
del tree['H']
del tree['K']
del tree['N']

# Parameter for draws 
n_nodes = {key1:{key3:len(tree[key1][key3]) for key3 in tree[key1].keys()} for key1 in tree.keys()}
n_groups = {key1:len(tree[key1]) for key1 in tree.keys()}

In [111]:
data.head()

,rome1,rome3,rome5,BE_id,T
0,A,A11,A1101,101,0.119945
1,A,A11,A1101,102,0.147218
2,A,A11,A1101,103,0.390912
3,A,A11,A1101,104,-0.118410
4,A,A11,A1101,105,-0.150113


Be careful !!! All draws that have strictly positive entries must come from a zeros=False draw with G as number of groups to redistribute. This problem arises as soon as one seeks to attribute G groups to n nodes with G > n (which could only happen here if we tried to redistribute groups at the top level. 

So if G > n ask for all zero draws with G-1 groups but delete among those groups those who have all strictly positive entries. (for instance drop draws if product > 0) 

In [ ]:
## VERSION 1
treeA = {key:values for key, values in tree.items() if key ==  'A'}
for key1, keys3 in treeA.items(): 
    G = len(keys3)
    print(key1,G)
    sigma = float('Inf')
    for part in partition(list(tree[key1].keys())):
        zeros = [] 
        lones = []
        for keys3 in part: 
            if len(keys3) > 1 : zeros.append(keys3)
            if len(keys3) == 1: lones.append(keys3[0])
        g = G - len(zeros)
        if g >= 0 :
            groups = {}
            i = 0 
            for keys3 in zeros: 
                i += 1
                groups[f'g{i}'] = []
                for key3 in keys3:
                    groups[f'g{i}'] += tree[key1][key3]
            max_lones = [len(tree[key1][key3]) for key3 in lones]
            nb_lones = len(lones)  
            if 0 < nb_lones <= g <= sum(max_lones):
                print(nb_lones,g,max_lones)
                draws3 = urns_and_balls(nb_lones,g,urns_max = max_lones,zeros = False,urns_names=lones)
                sigma3 = float('Inf')
                best_groups3 = []
                for draw3 in draws3: 
                    groups3 = []
                    others = {}
                    for key3 in lones:
                        if draw3[key3] == 1: groups3.append(tree[key1][key3])
                        if draw3[key3] > 1: others[key3] = draw3[key3]
                    best_groups5 = []
                    for key3, nb_groups in others.items():
                        if nb_groups == n_nodes[key1][key3]:
                            best_groups5 = [[rome5] for rome5 in tree[key1][key3]]
                        else:
                            best_groups5 = []
                            sigma5 = float('Inf')
                            for part5 in partition(tree[key1][key3]):
                                if len(part5) == nb_groups:
                                    part5_var = sum([data.loc[data['rome5'].isin(x)]['T'].var() for x in part5])
                                    if part5_var < sigma5: 
                                        sigma5 = part5_var
                                        best_groups5 = part5
                    groups3 += best_groups5  
                    groups3_var = sum([data.loc[data['rome5'].isin(group)]['T'].var() for group in groups3])
                    if groups3_var < sigma3:
                        sigma3 = groups3_var
                        best_groups3 = groups3
                for group in best_groups3:
                    i += 1
                    groups[f'g{i}'] = group
            part_var = sum([data.loc[data['rome5'].isin(group)]['T'].var() for group in groups.values()])
            if part_var < sigma:
                sigma = part_var
                best_groups = groups
    final_groups[key1] = best_groups

In [86]:
balls_max >= balls_min

True

In [124]:
## VERSION 2
treeA = {key:values for key, values in tree.items() if key ==  'A'}
final_groups = {}
for key1, keys3 in treeA.items():
    G = len(keys3)
    sigma_n = float('Inf')
    best_n = []
    for n in range(1,len(keys3)): # go to +1 ? 
        stops = [list(x) for x in combinations(keys3.keys(),n)]
        best_stops = []
        sigma_stops = float('Inf')
        for stop in stops:
            further = [key3 for key3 in keys3.keys() if key3 not in stop]
            further_nodes = {key3:len(tree[key1][key3]) for key3 in further}
            further_admissible = [len(tree[key1][key3]) > 1 for key3 in further]
            balls_max = [sum(further_nodes.values()),n]
            balls_min = [2*len(further),1]
            if len(stop) + sum(further_nodes.values()) > G and all(further_admissible) and sum(balls_min) <= G:
                draws = urns_and_balls(2,G,urns_max= balls_max,urns_min = balls_min,urns_names=['further','stop'],zeros=False)
                draws = [x for x in draws if len(x) > 0]
                best_draw = []
                sigma_draw = float('Inf')
                for draw in draws:
                    
                    # Minimise further within variance 
                    further_balls_max = list(further_nodes.values())
                    further_balls_min = [2 for key3 in further]
                    further_draws = urns_and_balls(len(further),
                                                   draw['further'],
                                                   urns_max= further_balls_max,
                                                   urns_min = further_balls_min, 
                                                   urns_names=further, 
                                                   zeros=False)
                    further_draws = [x for x in further_draws if len(x) > 0]
                    best_further_groups = []
                    sigma_further = float('Inf')
                    for further_draw in further_draws:
                        for key3 in further:
                            sigma_tot = 0 
                            further_groups = []
                            if len(tree[key1][key3]) == further_draw[key3]:
                                best_key_groups = [[rome5] for rome5 in tree[key1][key3]]
                                
                            else:
                                key_draws = urns_and_balls(further_nodes[key3],
                                                           further_draw[key3]-1,
                                                           urns_max=np.ones(further_nodes[key3],dtype=int),
                                                           zeros=True,
                                                           urns_names=tree[key1][key3])
                                best_further_groups = []
                                sigma_key = float('Inf')
                                for key_draw in key_draws:
                                    key_groups = []
                                    key_groups.append([rome for rome, nb in key_draw.items() if nb == 0])
                                    key_groups = [x for x in key_groups if len(x) > 0]
                                    for rome, nb in key_draw.items():
                                        if nb == 1: key_groups.append([rome])
                                    key_var = sum([data.loc[data['rome5'].isin(group)]['T'].var() for group in key_groups])
                                    if key_var < sigma_key: 
                                        sigma_key = key_var
                                        best_key_groups = key_groups
                            further_groups += best_key_groups
                            sigma_tot += sum([data.loc[data['rome5'].isin(group)]['T'].var() for group in best_key_groups])
                        if sigma_tot < sigma_further: 
                            sigma_further = sigma_tot
                            best_further_groups = further_groups
        
                    # Minimise STOP variance
                    parts = [x for x in partition(stop) if len(x) == draw['stop']]
                    sigma_part = float('Inf')
                    best_part = []
                    for part in parts:
                        var_part = sum([data.loc[data['rome3'].isin(x)]['T'].var() for x in part])
                        if var_part < sigma_part: 
                            sigma_part = var_part
                            best_part = part
                    best_part_groups = []
                    for group in best_part:
                        new_group = []
                        for key3 in group:
                            new_group += [rome for rome in tree[key1][key3]]
                        best_part_groups.append(new_group)
                    sigma_part += sigma_further
                    if sigma_part < sigma_draw:
                        sigma_draw = sigma_part
                        best_draw = best_part_groups + best_further_groups
                    print(draw.values())
                    print(len(best_further_groups),len(best_part_groups))
                if sigma_draw < sigma_stops: 
                    sigma_stops = sigma_draw
                    best_stops = best_draw
            if sigma_stops < sigma_n: 
                sigma_n = sigma_stops
                best_n = best_stops
        i = 0
        final_groups[key1] = {}
        for group in best_n:
            i += 1
            final_groups[key1][f'g{i}'] = group
        
            
                
                    
                        
                        
                            

                                        
                                        
                                    
                    
        

dict_values([4, 1])
2 1
dict_values([4, 1])
2 1
dict_values([4, 1])
2 1
dict_values([4, 1])
2 1
dict_values([4, 1])
2 1
dict_values([4, 1])
2 1
dict_values([2, 3])
2 3
dict_values([3, 2])
3 2
dict_values([4, 1])
4 1
dict_values([2, 3])
2 3
dict_values([3, 2])
3 2
dict_values([4, 1])
4 1
dict_values([2, 3])
2 3
dict_values([3, 2])
3 2
dict_values([2, 3])
2 3
dict_values([3, 2])
3 2
dict_values([4, 1])
4 1


In [122]:
final_groups

{'A': {'g1': ['A1101',
   'A1201',
   'A1202',
   'A1203',
   'A1204',
   'A1205',
   'A1501',
   'A1502',
   'A1503',
   'A1504'],
  'g2': ['A1401',
   'A1402',
   'A1403',
   'A1404',
   'A1405',
   'A1406',
   'A1407',
   'A1408',
   'A1409',
   'A1410',
   'A1411',
   'A1412',
   'A1413',
   'A1414',
   'A1415',
   'A1416'],
  'g3': ['A1417']}}

In [41]:




            for draw in gen_draws:
                #print('Trying to allocate ', draw)
                groups = {}
                i = 0
                # FURTHER 

                gen_draws_further = (x for x in draws_further if len(x) > 0)
                sigma_further = float('Inf')
                best_further = []
                for draw_further in gen_draws_further:
                    # select the best partition for each key3 of draw3
                    best_part5 = {}
                    sigmas = {key3:float('Inf') for key3 in further}
                    for key3 in further: 
                        parts5 = (x for x in partition(tree[key1][key3]) if len(x) == draw3[key3])
                        for part5 in parts5:
                            part5_var = sum([data.loc[data['rome5'].isin(x)]['T'].var() for x in part5])
                            if part5_var < sigmas5[key3]: 
                                sigmas5[key3] = part5_var
                                best_part5[key3] = part5
                    if sum(sigmas.values()) < sigma_further:
                        sigma_further = sum(sigmas.values())
                        best_further = best_part5
                print('this is the best_further',best_further)
                for group in best_further:
                    i += 1
                    groups[f'g{i}'] = group
                    
                # STOP 
                parts3 = (x for x in partition(stop) if len(x) == draw['stop'])
                sigma3 = float('Inf')
                best_part3 = []
                for part3 in parts3:
                    part3_var = sum([data.loc[data['rome3'].isin(x)]['T'].var() for x in part3])
                    if part3_var < sigma3: 
                        sigma3 = part3_var
                        best_part3 = part3
                for group in best_part3:
                    i += 1
                    groups[f'g{i}'] = group
                groups_var = sum([data.loc[data['rome5'].isin(group)]['T'].var() for group in groups.values()])
                
                if groups_var < sigma:
                    sigma = groups_var
                    best_groups = groups
    final_groups[key1] = best_groups
            
            
                    
                
            
                

                
                        

{'g1': 'A15', 'g2': ['A11'], 'g3': ['A12', 'A13']}

In [26]:
part5

[['A1412'],
 ['A1401',
  'A1402',
  'A1403',
  'A1404',
  'A1405',
  'A1406',
  'A1407',
  'A1408',
  'A1409',
  'A1410',
  'A1411',
  'A1413',
  'A1414',
  'A1415',
  'A1416',
  'A1417']]

In [149]:
draws3 = {key:urns_and_balls(n_groups[key],
                           n_groups[key]-1,
                           urns_max=list(n_nodes[key].values()),
                           urns_names=tree[key].keys(),
                           zeros=True) \
          for key in tree.keys()}

In [150]:
final_groups = {}
for key1 in tree.keys():
    print(key1)
    sigma = float('Inf')
    best_groups = {}
    for draw3 in draws3[key1]:
        zeros = []
        ones = []
        others = {}
        groups = {}
        i = 0
        for key3, nb_groups in draw3.items():
            if nb_groups == 0: zeros.append(key3)
            if nb_groups == 1: ones.append(key3)
            if nb_groups > 1 : others[key3] = nb_groups
        if len(zeros) > 0: groups[f'g{i}'] = []
        for key3 in zeros:
            for rome5 in tree[key1][key3]:
                groups[f'g{i}'].append(rome5)
        for key3 in ones:
            i += 1
            groups[f'g{i}'] = tree[key1][key3]
        for key3, nb_groups in others.items():
            if nb_groups == n_nodes[key1][key3]:
                best_groups5 = [[rome5] for rome5 in tree[key1][key3]]
            else:
                draws5 = urns_and_balls(n_nodes[key1][key3],
                                        nb_groups-1,
                                        urns_max=np.ones(n_nodes[key1][key3],dtype=int),
                                        zeros=True,
                                        urns_names=tree[key1][key3])
                best_groups5 = []
                sigma5 = float('Inf')
                for draw5 in draws5:
                    groups5 = []
                    groups5.append([rome5 for rome5, nb in draw5.items() if nb == 0])
                    groups5 = [x for x in groups5 if len(x) > 0]
                    for rome5, nb in draw5.items():
                        if nb == 1: groups5.append([rome5])
                    groups5_var = sum([data.loc[data['rome5'].isin(group)]['T'].var() for group in groups5])
                    if groups5_var < sigma5: 
                        sigma5 = groups5_var
                        best_groups5 = groups5
            for group in best_groups5:
                i += 1
                groups[f'g{i}'] = group
        groups_var = sum([data.loc[data['rome5'].isin(group)]['T'].var() for group in groups.values()])
        if groups_var < sigma:
            sigma = groups_var
            best_groups = groups
            best_draw = draw3
    final_groups[key1] = best_groups

A
B
C
D
E
F
G
I
J
L
M


In [151]:
final_output = {}
i = 0
for key1, groups1 in final_groups.items():
    for groups5 in groups1.values():
        i += 1
        for rome5 in groups5:
            final_output[rome5] = i
with open('optimal_grouping.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in final_output.items():
       writer.writerow([key, value])